# Lecture 1 Iterators

How do we compute something over an infinitely big input data structure? The answer to this question underpins the design of programming languages and frameworks for working with large datasets. 

## Countable Collections

The mathematical notion of infinity is complex, and oddly enough, there are *degrees* of infinity. We are interested in objects that are infinitely big. Consider the following mathematical sets:
* The set of real numbers $\mathbb{R}$ 
* The set of natural numbers $\mathbb{N}$ 
* The set of numbers divisible by 3 and 17 $\{51, 72, 153,...\}$

What do all of these sets have in common? First, all of these sets are infinitely big. Next, they are all ordered, i.e., for any two elements $e_1$ and $e_2$ there is a precise $>,<,=$ relationship. 

However, we intuitively feel that $\mathbb{R}$ is bigger than the other two sets. One aspect that is different between $\mathbb{R}$ and the other two sets is a clear definition of the "next" element. For example, if we take a real number $1.01$ what is the next real number? Is it $1.010001$ or $1.01000001$? We can go smaller *ad infinitum* making this process a fool's errand. 

This intuition gets at the notion of *countability*. A countable set is one that can be laid out in a precise sequence, where every element is assigned a location (e.g., the 5th element, or the 131st element):
$$[0,1,2,3,4,...]$$
$$[51, 72, 153,...]$$
In this sense, every countable set is indexed by the natural numbers. There is a unique correspondence between a position (described by a natural number) and an element. In summary, countability requires two conditions, a unique first element and for each element a unique next element. By induction, this creates a sequence like the ones above. 

Fun facts (prove them on your own):
* Every infinite countable set has the same cardinality (size) as $\mathbb{N}$.
* The set of rational numbers is countable
* All finite sets are countable

## Iterators
An iterator is a programatic way of interacting with countably "infinite" data. Infinite is in quotes because we know that any realistic dataset will not be infinite but *you are programming as if the dataset were infinite*. Rather than loading the entire input into a program, we process the input element-by-element. The design of iterators in programming languages mirrors the mathematical definitions of countability described above. 

Let's start with some simple examples in Python. The iteration paradigm that most are used to is the "for each" loop. For example, suppose I have a Python list:

In [1]:
lst = ['a', 'b', 'c']
for elem in lst:
    print(elem)

a
b
c


Python further allows one to explicitly track the indices of each fetched element of the list. Notice, how this definition mirrors the mathematical description above. We are assigning a natural number to each element of the list.

In [3]:
lst = ['a', 'b', 'c']
for index, elem in enumerate(lst):
    print(index, elem)

0 a
1 b
2 c


The list above is *materialized* that means it is explicitly stored in memory with all of its elements. Suppose, we wanted to enumerate an infinite set, say the Fibonacci sequence, we could not do that by hand. To be able to write programs that do this, we will have to get under the hood of how Python handles iteration and enumeration. 

As you have probably already noticed, we can only apply `for` loops to certain objects in Python. This is because only some classes in Python are *iterable*. An iterable class defines two properties `__iter__` and `__next__`, which initialize an iteration and return the next element---exactly mirroring the discussion about countability above! Let's first revist iterating over `lst` with the iterator API: 

In [4]:
lst = ['a', 'b', 'c']
lst_iter = iter(lst) #return the iterator object
print(next(lst_iter)) #'a'
print(next(lst_iter)) #'b'
print(next(lst_iter)) #'c'

a
b
c


When an iterator runs out of elements it raise a `StopIteration` exception:

In [5]:
print(next(lst_iter)) #'error'

StopIteration: 

Re-running `iter()` allows you to reset an iteration sequence:

In [6]:
lst = ['a', 'b', 'c']
lst_iter = iter(lst) #return the iterator object
print(next(lst_iter)) #'a'
print(next(lst_iter)) #'b'
lst_iter = iter(lst) #return the iterator object
print(next(lst_iter)) #'a'

a
b
a


## Writing your own iterators
This API is not that interesting for a materialized list, but is very powerful when we have to define complex sequences to iterate over. We do so by writing a class with the properties `__iter__` and `__next__`. Consider the following class that returns a sequence of every number divisible by 3.

In [9]:
class ThreeMultIterator():
    '''
    A class that iterates over multiples of three
    '''
    
    def __init__(self):
        '''
        Constructor, nothing here for now
        '''
        pass
    
    
    def __iter__(self):
        '''
        Initialize the iterator with the first element
        '''
        self.current = 0
        return self #have to return an object here!!
    
    def __next__(self):
        '''
        Returns the next element 
        '''
        
        elem = self.current #pin the current element
        self.current += 3 #update the current element
        
        return elem #return pin

The `ThreeMultIterator` class defines an infinite sequence of multiples of 3. To fetch the first 10 elements, we can write the following code:

In [10]:
threeMults = ThreeMultIterator()
three_iter = iter(threeMults)
limit = 10

while limit > 0:
    print(next(three_iter)) #print the next element
    limit -= 1 #decrement limit

0
3
6
9
12
15
18
21
24
27


Fortunately, you don't have to write code like this by hand. Python has a number of useful library functions to manipulate iterators. You can find these functions in the `itertools` module:

In [11]:
import itertools
threeMults = ThreeMultIterator()
three_iter = iter(threeMults)
print(list(itertools.islice(three_iter, 10))) #take the first 10 elements

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]


The important point to note is that `ThreeMultIterator` is an infinite sequence, however it can run just fine in Python with the iterator API. This is because the entire sequence is NOT materialized. We process just as much data as we need at each instant. Iterators can be more complex than simple numerical sequences, let's write an iterator that returns the Fibonacci series:
$$S_0 = 0$$
$$S_1 = 1$$
$$S_{i} = S_{i-1} + S_{i-2} \text{ for i > 1}$$

In [13]:
class Fib():
    '''
    A class that iterates over multiples of three
    '''
    
    def __init__(self):
        '''
        Constructor, nothing here for now
        '''
        pass
    
    
    def __iter__(self):
        '''
        Initialize the iterator with the first TWO elements
        '''
        self.i1 = 0
        self.i2 = 1
        self.current = 0
        
        return self #have to return an object here!!
    
    def __next__(self):
        '''
        Returns the next element 
        '''
        
        elem = self.current #pin the current element
        self.current = self.i1 + self.i2 #update the current element
        
        self.i2 = self.i1 #update the prev 2 elements
        self.i1 = self.current
       
        return elem #return pin

We can run similar code to that above to iterate over the infinite Fibonacci series

In [14]:
import itertools
fib = Fib()
fib_iter = iter(fib)
print(list(itertools.islice(fib, 10))) #take the first 10 elements

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


The Fibonacci example is interesting because the iterator needs to "remember" its previous return values. It does so through class variables. The class variables initialized and updated in `__iter__` and `__next__` are called state variables. 

## Practical Usage in Data Engineering
Going beyond the conceptual examples with numbers, let's discuss a concrete data engineering scenario. Often times we will be presented with a dataset that does not fit into main memory (but is stored as a file on a much bigger hard disk). For example, we might want to compute the total value (sum) of a very big file of numbers. We can use iterators to solve this problem. The file `my_file` contains a list of numbers. Let's write an iterator that "scans" this file:

In [16]:
class FileScan:
    """Loads a large file into the
    program line-by-line"""

    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        self.file = open(self.filename, 'r')
        self.line = self.file.readline()
        return self

    def __next__(self):
        if self.line != "":
            result = int(self.line)
            self.line = self.file.readline()
            return result
        else:
            self.file.close()
            raise StopIteration

We can write code to sum over all of the numbers:

In [17]:
scan = FileScan("my_file")
scan_iter = iter(scan)

total = 0
while True:
    try:
        total += next(scan_iter)
    except StopIteration:
        break

print(total)

234


We can use a for loop as shorthand for the above code snippet:

In [18]:
total = 0

for i in FileScan("my_file"):
    total += i
    
print(total)

234


What are the takeaway messages? Even though this file is very big the amount of "state" that we need to compute the sum is really only two numbers (the running total and the current number). 